# Applying score matching to real data
Using data from /nfs/nhome/live/maneesh/Work/motor-variability/Decoding1ms/MotorHam1.mat

Maneesh comments:
The matrix S contains spikes sampled at 1ms: Neuron x TimeBin x Trial.
Trials are shorter than size(S,3) -- so maybe restrict to first 1100
ms or so.  This is all one reach direction -- other MotorHam? files
have other reach directions.


In [1]:
using MAT # to load mat file
using NBInclude

In [2]:
# Load some data
file = matopen("MotorHam1.mat")
rawdata = read(file, "S")
size(rawdata) #neuron by timebin by trial

(199,1500,216)

In [3]:
# read the data into our input format, trial x (neuron) x ((spiketimes)))
hasfired = zeros(size(rawdata,1),1) # track if a neuron has fired ever
data = Array(Any, (size(rawdata,3),))
for n = 1:length(data)
    data[n] = Array(Any, (size(rawdata,1),))
    for m = 1:size(rawdata,1)        
        data[n][m] = find(x->(x>=1), rawdata[m,201:1200,n])*1e-3 # set of spiketimes between 0 and 1 sec
        if length(data[n][m]) > 10
            hasfired[m] = 1
        end
    end
end

# trim the data of neurons that are silent on every trial (no spikes whatsoever within the whole dataset)
for n = 1:length(data)
    data[n] = data[n][find(x->x==1, hasfired)]
end
        

In [4]:
# Load the processing files
nbinclude("ScoreMatching_functions.ipynb")

coordinate_descent (generic function with 1 method)

In [5]:
# Setup score matching, ex_dims = [M, D, F, N]. 
ex_dims = [1, 1, 16, 30]
rand_params = create_rand_params(ex_dims...); # Assumes T=1 (true) and ω=1 (true)
rand_params.T = 1000*1e-3
rand_params.ω = 0.25./rand_params.T
rand_params.C = rand_params.C
rand_params.θ = rand_params.θ
;

In [6]:
# Run coordinate descent

par = getall(rand_params);

penalise_integrals = Array(Any, 4)
penalise_integrals[2] = 1e2;
penalise_integrals[3] = Dict(:reltol=>1e-2, :abstol=>0.3);

optim_params = Dict()
optim_params[:iterations] = 10

par, S = coordinate_descent(data, par, max_iter=15, verbose=0, penalise_integrals=penalise_integrals, optim_params=optim_params);
#par, S = coordinate_descent(data, par, max_iter=3, verbose=1, penalise_integrals=0);

(N,M,D,F) = (30,1,1,16)


In [7]:
using JLD
save("MotorHam1_results_single_neur_6.jld", "par", par, "S", S, "data", data, "rand_params", rand_params, "penalise_integrals", penalise_integrals)

In [6]:
using JLD
tmp = load("MotorHam1_results_moderate.jld")
par = tmp["par"]
rand_params = tmp["rand_params"]
out = tmp["data"];
C, θ, ϕ, b, ω, N, M, D, F = name_params(par);
@show N,M,D,F

(N,M,D,F) = (10,20,2,8)


(10,20,2,8)

In [8]:
# Plot the results
# Plot data, true rate function and estimated rate function for a neuron on given trials
using PlotlyJS

num_neur = 1
trials = 11:15
dim_latent = 1

#Plot estimated
to_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
rate_rand_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
rate_est_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
latent_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
latent_est_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
colors = ["rgb(0,0,255)", "rgb(0,255,0)", "rgb(255,0,0)", "rgb(128,128,0)", "rgb(0,128,128)"]
for i1 = trials
    #@show collect(data[i1][num_neur])
    # Plot data
    to_plot[i1] = scatter(;x=collect(data[i1][num_neur]), y=i1*collect(ones(size(data[i1][num_neur]))), mode="markers", marker_color=colors[mod(i1,5)+1], yaxis="y2")
    
    # Compute underlying rate and latent functions
    rate_est = lambda_functions(0:0.001:rand_params.T, par[1], par[2]; b=par[4], phi=par[3][:,i1], omega=par[5])
    rate_rand = lambda_functions(0:0.001:rand_params.T, rand_params.C, rand_params.θ; b=rand_params.b, phi=rand_params.ϕ[:,i1], omega=rand_params.ω)
    latent_est = latent_functions(0:0.001:rand_params.T, par[2]; phi=par[3][:,i1], omega=par[5])
    
    # Plot underlying functions
    rate_est_plot[i1] = scatter(; 
    x=collect(0:0.001:rand_params.T), 
    y=collect(rate_est[num_neur,:]),
    line_color = colors[mod(i1,5)+1],
    yaxis = "y0"
)

    rate_rand_plot[i1] = scatter(; 
    x=collect(0:0.001:rand_params.T), 
    y=collect(rate_rand[num_neur,:]),
    line_color = colors[mod(i1,5)+1],
    yaxis = "y0"
)

    latent_est_plot[i1] = scatter(; 
    x=collect(0:0.001:rand_params.T), 
    y=collect(latent_est[dim_latent,:]),
    line_color = colors[mod(i1,5)+1],
    yaxis = "y0"
)

end

lo = Layout(;xaxis_range=[0,rand_params.T])

#plt = plot([Plot(to_plot[trials]); Plot(rate_plot[trials]);  Plot(rate_est_plot[trials]);  Plot(latent_plot[trials]);  Plot(latent_est_plot[trials])])
plt = plot([Plot(to_plot[trials], lo); Plot(rate_est_plot[trials], lo); Plot(rate_rand_plot[trials],lo)])

relayout!(plt, height=700)

plt

In [ ]:
par[2]'

In [166]:
# Show number of spikes, integral of true rate function and integral of estimated rate function
for j1 = 1
for i1=1:ex_dims[4]
        bla_est = lambda_functions(0:0.001:rand_params.T, par[1], par[2]; b=par[4], phi=par[3][:,i1], omega=par[5])

        a= [size(data[i1][j1],1)/rand_params.T (sum(bla_est[j1,:])/length(bla_est[j1,:]))]
    @show a
    end
end

a = [18.0 18.002563733193682]
a = [14.0 13.999858959210039]
a = [18.0 17.98195532353987]
a = [21.0 21.042142666818208]
a = [17.0 17.02764366295114]
a = [17.0 17.075462408891852]
a = [7.0 8.025271731366642]
a = [10.0 10.011147788818166]
a = [18.0 18.024988019217858]
a = [18.0 18.005039239459318]
a = [15.0 15.019325730373728]
a = [17.0 17.017696938832724]
a = [19.0 18.992908604281432]
a = [13.0 13.052623917416584]
a = [11.0 11.003048157640217]
a = [11.0 11.067301821312574]
a = [7.0 8.141738572539941]
a = [10.0 10.057113452790471]
a = [14.0 14.006682306612571]
a = [16.0 16.034158901178802]
a = [17.0 17.01694387603135]
a = [22.0 22.040923208390346]
a = [17.0 16.99424300176361]
a = [13.0 12.989151430298607]
a = [13.0 12.992663504481438]
a = [12.0 11.950435048735434]
a = [10.0 9.982348033522346]
a = [9.0 9.003590747174911]
a = [13.0 13.051538483540087]
a = [13.0 13.05648790465824]


In [44]:
ddexp(par[4])

1x1 Array{Float64,2}:
 14.1

In [64]:
num_neur = 1
[((par[1]*par[2])[num_neur,:]').^2 rand_params.ω*(1:ex_dims[3])] # How much power from each frequency gets mapped from to the neuron

LoadError: LoadError: ArgumentError: number of rows of each array must match (got (8,16))
while loading In[64], in expression starting on line 2

In [167]:
plot(scatter(;x=(1:ex_dims[3])*par[5], y=((par[1]*par[2])[:]).^2))